# Connection à la base de données 'fcd'

In [1]:
import pandas as pd
import pymongo

client = pymongo.MongoClient('localhost')
collection = client.fcd.assets

print('done')

done


# Lecture dans la base

In [2]:
cursor = collection.find()
entries = list(cursor)

#entries[:]

df = pd.DataFrame(entries)
df.head()
print('\ndone')

,_id,asset_id,recorded_at,location
0,5f16a5bd9c984c3e2ad2fc23,14429000032073,1.583104e+09,"{'geo': {'coordinates': [-114.028, 50.9551], '..."
1,5f16a5bd9c984c3e2ad2fc24,14429000032073,1.583104e+09,"{'geo': {'coordinates': [-114.028, 50.9554], '..."
2,5f16a5bd9c984c3e2ad2fc25,14429000032073,1.583104e+09,"{'geo': {'coordinates': [-114.028, 50.9558], '..."
3,5f16a5bd9c984c3e2ad2fc26,14429000032073,1.583104e+09,"{'geo': {'coordinates': [-114.026, 50.9554], '..."
4,5f16a5bd9c984c3e2ad2fc27,14429000032073,1.583104e+09,"{'geo': {'coordinates': [-114.026, 50.9544], '..."


# Entrée 1 : géo

In [2]:
lon = float(input("longitude -> "))
lat = float(input("latitude -> "))
rad = float(input("rayon (km) -> "))

print('\ndone')

longitude -> -113.584
latitude -> 53.5124
rayon (km) -> 5

done


# Entrée 2 : date

In [3]:
from datetime import timedelta
from datetime import datetime

year = int(input("Annee -> "))
month = int(input("Mois -> "))
day = int(input("Jour -> "))
hour = int(input("Heure -> "))
minutes = int(input("Minutes -> "))
seconds = int(input("Secondes -> "))
date = datetime(year, month, day, hour, minutes, seconds)

print('\ndone')

Annee -> 2020
Mois -> 3
Jour -> 5
Heure -> 14
Minutes -> 30
Secondes -> 5

done


# Entrée 3 : période

In [4]:
T = int(input("Période (minutes) -> "))
delta = timedelta(minutes = float(T/2))

datemin = date - delta
datemax = date + delta

datemin = datetime.timestamp(datemin)
datemax = datetime.timestamp(datemax)

print('\ndone')

Période (minutes) -> 60

done


# QUERY 1 

In [5]:
import pprint


# QUERY 1
# Calculer la densité [nombre de points] sur une Zone géographique Z et une période T 

def density_list(lon, lat, rad, datemin, datemax, option):
    
    # points dans une zone Z
    query01 = {
        "location.geo": {
            "$nearSphere": {
                "$geometry": {
                    "type": "Point",
                    "coordinates": [ lon, lat]
                },
                "$minDistance": 0,
                "$maxDistance": rad*1000 # mètres
            }
        }
    }

    # points à une période T
    query02 = {'$and': [{'recorded_at': {"$gt": datemin}}, {'recorded_at': {"$lt": datemax}}]}


    # requête finale
    query = {'$and': [query01, query02]}

    if option != {} :
        query = {'$and': [query, option]}

    return list(collection.find(query))


result = density_list(lon, lat, rad, datemin, datemax, {})
print ("density :", len(result))

print('done')

density : 0
done


# QUERY 2

In [6]:
import pprint

# QUERY 2
# Retourner le nombre de points voisins d'un asset X dans une période T.


def asset_density(id_asset, rad, datemin, datemax):

    # les points d'un asset X
    query01 = {"asset_id": id_asset}

    # points à une période T
    query02 = {'$and': [{'recorded_at': {"$gt": datemin}}, {'recorded_at': {"$lt": datemax}}]}

    # les points d'un asset X à une période T
    query03 = {'$and': [query01, query02]}

    tmp = list(collection.find(query03))

    lon = 0
    lat = 0
    density = 0
    result = {}
    res = None

    for doc in tmp:
        lon = doc['location']['geo']['coordinates'][0]
        lat = doc['location']['geo']['coordinates'][1]
        d_list = density_list(lon, lat, rad, datemin, datemax, {"asset_id": {"$ne": id_asset}})

        if len(d_list) > density:
            density = len(d_list)
            res = doc

    if len(tmp) > 0 and density > 0:
        result['asset_id'] = id_asset
        result['density'] = density
        result['radius'] = rad
        result['center'] = res['location']['geo']['coordinates']

    return result
    
    
result = asset_density('14429000019112', rad, datemin, datemax)

if result != {}:
    print('density :', result['density'])
    print('circle zone of',result['radius'],'km around', result['center'])

print('\ndone')

density : 32
circle zone of 5.0 km around [-113.403, 53.4946]

done


# QUERY 3

In [8]:
# QUERY 3
# Retournez le top k des périodes/ zones les plus denses


def top_k_zones(k, rad, datemin, datemax):
    
    # list des id des différents asset
    ids_asset = list(collection.distinct("asset_id"))

    zones = []

    for id_ in ids_asset:

        zone = asset_density(id_, rad, datemin, datemax)

        if zone != {}:
            zones.append(zone)


    zones = sorted(zones, key=lambda zone:zone['density'], reverse=True)
    
    if k > len(zones):
        k = len(zones)
        
    return zones[0:k]

zones = top_k_zones(3, rad, datemin, datemax)
pprint.pprint(zones)
print('\ndone')

[{'asset_id': '14429000008909',
  'center': [-114.031, 50.9487],
  'density': 265,
  'radius': 5.0},
 {'asset_id': '14429000035001',
  'center': [-114.06, 50.8978],
  'density': 261,
  'radius': 5.0},
 {'asset_id': '14429000032073',
  'center': [-114.043, 50.8931],
  'density': 239,
  'radius': 5.0}]

done
